In [23]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import math


In [4]:


'''def clothesEdge(pose_points):
    image = cv2.imread("images/testImage.png")
    if image is None:
        print("Error: image not found")
        return
    #blurs the image to make the edges more prominent
    img_blur = cv2.GaussianBlur(image,(3,3), sigmaX=0, sigmaY=0)
    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_points = []
for contour in contours:
        e = 0.02 * cv2.arcLength(contour, True)
        simple_contour = cv2.approxPolyDP(contour, e, True)
    
    #pose_points = [left_s, right_s, left_h, right_h]
    for point in pose_points:
        distance = [np.linalg.norm(point - contour_point[0]) for contour_point in simple_contour]
        closest_index = np.argmin(distance)
        contour_points.append(simple_contour[closest_index][0])
        closest_point = None
        min_distance = float('inf')
        for contour in contours:
            e = 0.02 * cv2.arcLength(contour, True)
            simple_contour = cv2.approxPolyDP(contour, e, True)

            for contour_point in simple_contour:
                distance = np.linalg.norm(point - contour_point[0])
                if distance < min_distance:
                    min_distance = distance
                    closest_point = contour_point[0]

        if closest_point is not None:
            contour_points.append(closest_point)
        x, y, w, h = cv2.boundingRect(contour)
        contour_points.append((x,y)) #top left
        contour_points.append((x+w,y)) #top right
        contour_points.append((x+w,y+h)) #bottom right
        contour_points.append((x,y+h)) #bottom left

        for point in contour:
            cv2.circle(image, (x,y), radius=2, color=(0, 255, 0), thickness=-1)

            x, y = point[0]
            #print(f"Contour point:({x},{y})")
            label = f"{x},{y}"
            cv2.putText(image, label, (x+5,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,0,0), 1, cv2.LINE_AA)
    contour_image = image.copy()
    cv2.drawContours(contour_image,contours,-1, (0,255,0), 2)
    
    #cv2.imshow("Canny edge", edges)
    #cv2.imshow("Contours", contour_image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #print(contour_points)
    return contour_points'''

In [5]:
def rotation_angle(pose_points):
    shoulder_center = np.mean([pose_points[0], pose_points[1]], axis=0)
    hip_center = np.mean([pose_points[3], pose_points[2]], axis=0)

    delta_y = shoulder_center[1]
    delta_x = shoulder_center[0]

    angle = np.degrees(np.arctan2(delta_y,delta_x))
    return angle

In [19]:
def mapClothes(pose_points, img):
    
    fixedRatio = 262/190 # for shirt width
    heightToWidth = img.shape[0]/img.shape[1] #aspect ratio of image
    width = int(math.fabs(pose_points[1][0]-pose_points[0][0])*fixedRatio)
    height = int(width*heightToWidth)
    x_offset = 40
    src = np.array([[0,0],[width, 0], [width, height], [0,height]], dtype='float32')
    dest = np.array([[pose_points[0][0]+x_offset, pose_points[0][1]], [pose_points[1][0]-x_offset,\
                    pose_points[1][1]], [pose_points[2][0] - 2*x_offset, pose_points[2][1]],\
                         [pose_points[3][0]+ 2*x_offset, pose_points[3][1]]],dtype='float32')
    transform = cv2.getPerspectiveTransform(src,dest)
    
    '''print(f"Original image shape: {img.shape}")  # Print original shape
    print(f"Calculated width for resizing: {width}")'''
    angle = rotation_angle(pose_points)
    center = img.shape[1]//2, img.shape[0]//2
    scale = math.fabs(pose_points[1][0]-pose_points[0][0])/190
    mat = cv2.getRotationMatrix2D(center, -angle, 1.0)
    #img = cv2.warpAffine(img, mat, (img.shape[1], img.shape[0]))
    img = cv2.resize(img, (width, height))
    #print(f"Resized image shape: {img.shape}")
    offset = [int(44*scale), int(48*scale)] #horizontal and vertical offsets
    return img, transform
    

In [18]:
def main():
    img = cv2.imread('images/testImage.png')
    mapClothes([(340,220),(450,660),(220,320),(330,450)], img)
if __name__ == "__main__":
    main()